In [1]:
using ModelingToolkit
using ModelingToolkit: t_nounits as t, D_nounits as D
using JuMP: @variable, @objective, @constraint, Model, optimize!, termination_status, value
using CSV, DataFrames, Distributions
using Random, Dates, Unitful

In [2]:
include("./Influent/InfluentGenerator.jl");
include("./Effluent/EffluentGenerator.jl");
include("./Unit Operations/MaterialStream.jl");
include("./Unit Operations/Mixer.jl");
include("./Unit Operations/Splitter.jl");
include("./Unit Operations/CSTR.jl");

In [3]:
influent = generate_influent_ss();
inlet_temp = influent[1];
inlet_comp = influent[2:end];
# UConn average flowrate in m3 / day -> tau for each reactor = 0.3 days -> 1.5 days for whole BNR system
inlet_flowrate = 3785.42 
# ============================================= EFFLUENT DATA ================================================#
effluent = generate_effluent_ss();
effluent_comp = effluent[2:end];
# ============================================= SIMULATION TIME ==============================================#
# time of simulation in days
t_end = 100;

init_vec = copy(inlet_comp);
init_vec[10] = 5;
init_vec[11] = 5;
init_vec[12] = 5;

init_vec_anoxic = copy(init_vec);
init_vec_anoxic[1] = 0.0;


COD/BOD ratios saved to ./Influent/COD_BOD_ratios.csv
Number of common dates: 440
Number of rows in filtered ratios_df: 440
Number of rows in filtered df: 440


In [ ]:
function compile_BNR()

    @mtkmodel BNRPlantModel begin
        @components begin

            Influent = InletStream(comp = inlet_comp, flow_rate = inlet_flowrate)
            reactor1 = ASM3_CSTR_Aerator(vol = 1135.6, switch=0, Temp = 20)
            reactor2 = ASM3_CSTR_Aerator(vol = 1135.6, switch=1, Temp = 20)
            reactor3 = ASM3_CSTR_Aerator(vol = 1135.6, switch=0, Temp = 20)
            reactor4 = ASM3_CSTR_Aerator(vol = 1135.6, switch=1, Temp = 20)
            reactor5 = ASM3_CSTR_Aerator(vol = 1135.6, switch=0, Temp = 20)
            splitter1 = Splitter_1in_3out(Out_factor = [0.5,0.4,0.1])
            mixer1 = Mixer_2in_1out()
            mixer2 = Mixer_2in_1out()

        end
    
        @equations begin
            connect(Influent.port,mixer1.In1)
            connect(splitter1.Out3,mixer1.In2)
            connect(mixer1.Out1,reactor1.In)
            connect(reactor1.Out,mixer2.In1)
            connect(splitter1.Out1,reactor2.In)
            connect(reactor2.Out,mixer2.In2)
            connect(mixer2.Out1,reactor3.In)
            connect(reactor3.Out,reactor4.In)
            connect(reactor4.Out,reactor5.In)
            connect(reactor5.Out,splitter1.In1)
        end
    end

    @mtkcompile BNR_system = BNRPlantModel() 

    return BNR_system
end